<a href="https://colab.research.google.com/github/Gharshavardhankumar/Predict-Product-Backorders--Classification/blob/main/20220825_Batch121_CDS_7456_DecisionTreeClassifier_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the required libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

from imblearn.over_sampling import SMOTE

import graphviz

# Identify Right Error Metrics

    Based on the business have to identify the right error metrics.

## Function to calculate required metrics

In [ ]:
def evaluate_model(act, pred):
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred))
    print("Precision: ", precision_score(act, pred))    

# Loading the data

In [ ]:
data = pd.read_csv("/home/datasets/lab/DT_BackOrders.csv", header=0)

# Understand the Data - - Exploratory Data Analysis (EDA)

## Number row and columns

In [ ]:
data.shape

(61589, 23)

## First and last 5 records

In [ ]:
data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,1758220,46,2.0,0,0,0,0,1,2,6,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,1360312,2,2.0,0,4,6,10,2,2,5,...,0,0.97,0.92,0,No,No,No,Yes,No,No


In [ ]:
data.tail()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
61584,1397275,6,8.0,0,24,24,24,0,7,9,...,0,0.98,0.98,0,No,No,No,Yes,No,No
61585,3072139,130,2.0,0,40,80,140,18,108,230,...,0,0.51,0.28,0,No,No,No,Yes,No,No
61586,1909363,135,9.0,0,0,0,0,10,40,65,...,0,1.00,0.99,0,No,No,Yes,Yes,No,No
61587,1845783,63,NaN,0,0,0,0,452,1715,3425,...,0,-99.00,-99.00,1,No,No,No,No,No,Yes
61588,1200539,0,2.0,0,8,8,8,0,1,1,...,0,0.79,0.78,0,Yes,No,No,Yes,No,Yes


## Statistic summary 
    Using describe function

In [ ]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,6.158900e+04,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,...,61589.000000,61589.000000,61589.000000,61589.000000,61589,61589,61589,61589,61589,61589
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48145,61577,53792,59303,61569,50296
mean,2.037188e+06,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,...,1.605400,-6.264182,-5.863664,1.205361,NaN,NaN,NaN,NaN,NaN,NaN
std,6.564178e+05,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,...,42.309229,25.537906,24.844514,29.981155,NaN,NaN,NaN,NaN,NaN,NaN
min,1.068628e+06,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,-99.000000,-99.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.498574e+06,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.620000,0.640000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.898033e+06,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,...,0.000000,0.820000,0.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.314826e+06,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,...,0.000000,0.960000,0.950000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


## Data type

In [ ]:
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

__Observations__

* sku is `categorical` but is interpreted as `int64` 
* potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder are also `categorical` but is interpreted as `object`. 

# Data pre-processing

## Convert all the attributes to appropriate type

Data type conversion

    Using astype('category') to convert potential_issue, deck_risk, 
    oe_constraint, ppap_risk, stop_auto_buy, rev_stop, 
    and went_on_backorder attributes to categorical attributes.


In [ ]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
            'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')

### Re-display data type of each variable

In [ ]:
data.dtypes

sku                  category
national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

### Statistic summary 
    Using describe function

In [ ]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,61589.0,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,...,61589.000000,61589.000000,61589.000000,61589.000000,61589,61589,61589,61589,61589,61589
unique,61589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,3284895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48145,61577,53792,59303,61569,50296
mean,NaN,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,...,1.605400,-6.264182,-5.863664,1.205361,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,...,42.309229,25.537906,24.844514,29.981155,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,-99.000000,-99.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.620000,0.640000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,...,0.000000,0.820000,0.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,...,0.000000,0.960000,0.950000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


## Delete sku attribute

In [ ]:
np.size(np.unique(data.sku, return_counts=True)[0])

61589

In [ ]:
data.drop('sku', axis=1, inplace=True)

## Missing Data

    Missing value analysis and dropping the records with missing values

In [ ]:
data.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

Observing the number of records before and after missing value records removal

In [ ]:
print (data.shape)

(61589, 22)


Since the number of missing values is about 5% and as we have around 61K records. For initial analysis we ignore all these records

In [ ]:
data = data.dropna(axis=0)

In [ ]:
print(data.isnull().sum())
print(data.shape)

national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64
(58186, 22)


## Encoding Categorical to Numeric

`pandas.get_dummies` To convert convert categorical variable into dummy/indicator variables

__Creating dummy variables__

If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category.

In [ ]:
cat_attr_names = data.select_dtypes(include=['category']).columns

In [ ]:
data = pd.get_dummies(columns=cat_attr_names, data=data, 
                      prefix=cat_attr_names, prefix_sep="_", drop_first=True)

In [ ]:
print (data.columns, data.shape)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty', 'potential_issue_Yes', 'deck_risk_Yes',
       'oe_constraint_Yes', 'ppap_risk_Yes', 'stop_auto_buy_Yes',
       'rev_stop_Yes', 'went_on_backorder_Yes'],
      dtype='object') (58186, 22)


## Train and test split

### Target attribute distribution 

In [ ]:
data['went_on_backorder_Yes'].value_counts()

0    47217
1    10969
Name: went_on_backorder_Yes, dtype: int64

In [ ]:
data['went_on_backorder_Yes'].value_counts(normalize=True)*100

0    81.148386
1    18.851614
Name: went_on_backorder_Yes, dtype: float64

### Split the data into train and test

`sklearn.model_selection.train_test_split`

    Split arrays or matrices into random train and test subsets

In [ ]:
X = data.drop('went_on_backorder_Yes', axis=1)
y = data['went_on_backorder_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123) 

### Target attribute distribution after the split

In [ ]:
print(pd.value_counts(y_train, normalize=True) * 100)

print(pd.value_counts(y_test, normalize=True) * 100)

0    81.185858
1    18.814142
Name: went_on_backorder_Yes, dtype: float64
0    81.060953
1    18.939047
Name: went_on_backorder_Yes, dtype: float64


# Model building

## Decision Tree Model

### Instantiate Model

In [ ]:
dtclf = DecisionTreeClassifier()

### Train Model

In [ ]:
dtclf.fit(X_train, y_train)

DecisionTreeClassifier()

### List important features

In [ ]:
importances = dtclf.feature_importances_
importances

array([0.31291385, 0.03169478, 0.0311396 , 0.0912365 , 0.02871705,
       0.1107907 , 0.07051742, 0.03497093, 0.05016168, 0.04610257,
       0.03033654, 0.00721194, 0.05746401, 0.06297542, 0.01063403,
       0.00039911, 0.01149081, 0.        , 0.00912832, 0.00211474,
       0.        ])

In [ ]:
indices = np.argsort(importances)[::-1]
ind_attr_names = X_train.columns
pd.DataFrame([ind_attr_names[indices], np.sort(importances)[::-1]])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,national_inv,forecast_9_month,forecast_3_month,sales_1_month,perf_12_month_avg,perf_6_month_avg,sales_6_month,sales_9_month,sales_3_month,lead_time,...,min_bank,forecast_6_month,deck_risk_Yes,local_bo_qty,ppap_risk_Yes,pieces_past_due,stop_auto_buy_Yes,potential_issue_Yes,oe_constraint_Yes,rev_stop_Yes
1,0.312914,0.110791,0.0912365,0.0705174,0.0629754,0.057464,0.0501617,0.0461026,0.0349709,0.0316948,...,0.0303365,0.028717,0.0114908,0.010634,0.00912832,0.00721194,0.00211474,0.000399111,0,0


In [ ]:
train_pred = dtclf.predict(X_train)
test_pred = dtclf.predict(X_test)

In [ ]:
def evaluate_model(act, pred):
    from sklearn.metrics import confusion_matrix, accuracy_score,\
     recall_score, precision_score
    
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred))
    print("Precision: ", precision_score(act, pred))   

### Evaluate

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[33047    20]
 [   69  7594]]
Accurcay :  0.9978148784679597
Recall   :  0.9909956935925878
Precision:  0.9973732597846073
--Test--
Confusion Matrix 
 [[13223   927]
 [  770  2536]]
Accurcay :  0.9027841429880843
Recall   :  0.7670901391409558
Precision:  0.7323130233901242


### Observation:
    Recall is pretty low

__Reason__:

    Class imbalance i.e. 81% of the records are not back order records 

## Up-sampling 

    Using SMOTE (Synthetic Minority Over-sampling Technique)

### Instantiate SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=123)

### Fit Sample

In [ ]:
X_train_sm, y_train_sm = smote.fit_sample(X_train, y_train)

In [ ]:
X_train_sm.shape

(66134, 21)

In [ ]:
y_train_sm.shape

(66134,)

In [ ]:
print(pd.value_counts(y_train_sm, normalize=True) * 100)

1    50.0
0    50.0
Name: went_on_backorder_Yes, dtype: float64


## Decision Tree with up-sample data

### Instantiate Model

In [ ]:
dtclf2 = DecisionTreeClassifier()

### Train the model

In [ ]:
dtclf2 = dtclf2.fit(X_train_sm, y_train_sm)

### List important features

In [ ]:
importances = dtclf2.feature_importances_
importances

array([0.28368106, 0.        , 0.        , 0.71631894, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [ ]:
indices = np.argsort(importances)[::-1]
pd.DataFrame([ind_attr_names[indices], np.sort(importances)[::-1]])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,forecast_3_month,national_inv,sales_9_month,lead_time,in_transit_qty,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,stop_auto_buy_Yes,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,potential_issue_Yes,deck_risk_Yes,oe_constraint_Yes,ppap_risk_Yes,min_bank
1,0.716319,0.283681,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Predict

In [ ]:
train_pred=dtclf2.predict(X_train_sm)
test_pred=dtclf2.predict(X_test)

### Evaluate

In [ ]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[28025  5042]
 [ 6412 26655]]
Accurcay :  0.8268061813893005
Recall   :  0.8060906644086249
Precision:  0.8409313184213018
--Test--
Confusion Matrix 
 [[11949  2201]
 [  656  2650]]
Accurcay :  0.8363313473877176
Recall   :  0.8015728977616455
Precision:  0.5462791177076891
